In [2]:
import numpy as np
import pandas as pd
from toolbox import watts2dBm, dBm2watts, dB2lin, lin2dB, pathloss, pack2dict
from antenna import AntennaArray, Omnidirectional
from simulation import simulate
import time 
import os 
from multiprocessing import Process
import logging
import argparse
#
logging.basicConfig(filename="../logs/ris_size.log", encoding="utf-8", level=logging.INFO)

#### Hyperparameters of the simulation.

In [3]:
'''
Network topology (coordinates <x, y, z>).
'''
bs_loc  = np.array([0, 0, 0]) # Base station.
ris_loc = np.array([5, 5, 0]) # RIS.
ue_loc  = np.array([50, 0, 0]) # User equipment.
'''
Channel parameters.
'''
Ptx_dBm = 25
A_dB = -30 # 30 dB path loss attenuation.
# BS -> RIS -> UE is LOS; BS -> UE is NLOS.
alpha_bs2ris = 2.8 # Attenuation coefficient between base station and RIS.
alpha_ris2ue = 2.8 # ... between RIS and user equipment.
# alpha_bs2ue  = 3.5 # ... between BS and user equipment.
'''
Tx antenna, i.e., base station, considering the downlink communication.
'''
M = 8 # Number of antenna elements.
fc = 30e9 # Carrier frequency.
wavelen = 3e8/fc # Wavelength of the 30 GHz carrier frequency.
nx, ny, nz = 1, M, 1 # Antenna array (3D?).
dx = dy = dz = wavelen * 0.5
bs_antenna = AntennaArray(wavelen, nx, ny, nz, dx, dy, dz)
'''
Quantization.
'''
nbits = 8

### Simulation.

In [7]:
def run_sim(niter, nris, alpha_bs2ue, savefile=None):
    '''
    This function performs the RIS-aided attack ``niter'' times with a RIS of 
    size ``nris'' elements and direct link (BS -> UE) path loss exponent 
    ``alpha_bs2ue''.

    It either returns or saves (to ``savefile'' file) a data frame.
    '''
    seed = int.from_bytes(os.urandom(4), byteorder="little")
    np.random.seed(seed)

    results = {"opt": [], "nris": [], "alpha_bs2ue": [], "prx_dBm": []}
    for iter in range(niter):
        for opt in ["sota python", "random", "none"]:
            # np.random.seed(seed+iter)
            np.random.seed(iter)
            prx_dBm = simulate(
                Ptx_dBm=Ptx_dBm, 
                bs_loc=bs_loc, 
                ue_loc=ue_loc, 
                ris_loc=ris_loc, 
                bs_antenna=AntennaArray(wavelen, nx, ny, nz, dx, dy, dz), 
                # bs_antenna=Omnidirectional(wavelen, nx, ny, nz, dx, dy, dz),
                M=M, 
                N=nris, 
                alpha_bs2ue=alpha_bs2ue, 
                alpha_bs2ris=alpha_bs2ris, 
                alpha_ris2ue=alpha_ris2ue, 
                ris_optimizer=opt, 
                estimation_err_dB=-np.inf, 
                nbits=nbits,
                A_dB=A_dB,
            ) 
            logging.info(f"{opt}, n = {nris}, a = {alpha_bs2ue}, prx = {prx_dBm}.")
            _ = pack2dict(results, [opt, nris, alpha_bs2ue, prx_dBm])

    df = pd.DataFrame(results)

    if savefile is not None:    
        df.to_csv(savefile, index=False)
        return
    else:
        return df 

In [8]:
df = run_sim(niter=10, nris=200, alpha_bs2ue=3.5)
df.head()

,opt,nris,alpha_bs2ue,prx_dBm
0,sota python,200,3.5,-70.636766
1,random,200,3.5,-67.106558
2,none,200,3.5,-67.560521
3,sota python,200,3.5,-145.731332
4,random,200,3.5,-79.580823


### Data analysis.

In [9]:
df.groupby(["opt", "nris", "alpha_bs2ue"]).mean()

,,,prx_dBm
opt,nris,alpha_bs2ue,
none,200,3.5,-76.037480
random,200,3.5,-76.353167
sota python,200,3.5,-111.533380
